In [3]:
import datetime as dt
import pyarrow.feather as feather
import json
import pandas as pd
import pprint
import praw
import requests
from requests.exceptions import HTTPError

from google.cloud import storage
from psaw import PushshiftAPI

# Display options
pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 400)

In [2]:
# Import client id and secret to use with PRAW
with open("keys.json", "r") as credentials:
    data = json.load(credentials)
    c_id = data["client_id"]
    c_secret = data["client_secret"]
    u = data["username"]
    p = data["password"]

# Initialise Google Cloud Storage

In [4]:
# Create a client
client = storage.Client()
print("Client created using default project: {}".format(client.project))

# List available buckets
buckets = client.list_buckets()

# List bucket metadata
print("Buckets in {}:".format(client.project))
for bucket in buckets:
    print("\t Bucket name: {}".format(bucket.name))
    print("\t Bucket location: {}".format(bucket.location))
    print("\t Bucket storage class: {}".format(bucket.storage_class))

Client created using default project: tonal-studio-322315
Buckets in tonal-studio-322315:
	 Bucket name: talk-bucket
	 Bucket location: EUROPE-WEST2
	 Bucket storage class: STANDARD


In [5]:
# Set bucket for data storage
bucket = client.get_bucket('talk-bucket')

In [6]:
# Check current data storage utilisation in bucket
blobs = bucket.list_blobs()

print("Blobs in {}:".format(bucket.name))
for item in blobs:
    print("Name: " + item.name)
    print("\t ID: {}".format(item.id))
    print("\t Size: {} MB".format(item.size*0.000001))
    print("\t Content type: {}".format(item.content_type))
    print("\t Public URL: {}".format(item.public_url))
    print("\n")

Blobs in talk-bucket:
Name: AgainstHateSubreddits
	 ID: talk-bucket/AgainstHateSubreddits/1628511471991024
	 Size: 6.14297 MB
	 Content type: application/octet-stream
	 Public URL: https://storage.googleapis.com/talk-bucket/AgainstHateSubreddits


Name: GenderCritical
	 ID: talk-bucket/GenderCritical/1629192636392187
	 Size: 18.061418 MB
	 Content type: application/octet-stream
	 Public URL: https://storage.googleapis.com/talk-bucket/GenderCritical


Name: MGTOW
	 ID: talk-bucket/MGTOW/1629192130424207
	 Size: 52.368258 MB
	 Content type: application/octet-stream
	 Public URL: https://storage.googleapis.com/talk-bucket/MGTOW


Name: MGTOW2
	 ID: talk-bucket/MGTOW2/1629192252369299
	 Size: 6.459137999999999 MB
	 Content type: application/octet-stream
	 Public URL: https://storage.googleapis.com/talk-bucket/MGTOW2


Name: The_Donald0
	 ID: talk-bucket/The_Donald0/1629281818858464
	 Size: 26.395754 MB
	 Content type: application/octet-stream
	 Public URL: https://storage.googleapis.com/ta

# Data Collection

### Initialisation

In [7]:
# Initialise PRAW instance
reddit = praw.Reddit(
    client_id = c_id,
    client_secret = c_secret,
    user_agent = "my user agent",
    username = u,
    password = p,
)

# Initialise PSAW instance with PRAW 
api = PushshiftAPI(reddit)

# Initialise PSAW instance without PRAW for banned subreddits
api_banned = PushshiftAPI()

### Function for Data Collection

In [8]:
# Set start and end dates for data retrieval
start_epoch = int(dt.datetime(2018, 4, 1).timestamp())
end_epoch = int(dt.datetime(2021, 4, 30).timestamp())

In [9]:
# Function to retrieve and store a number of posts from a given subreddit
def get_posts(subreddit, 
              start_epoch = start_epoch, 
              end_epoch = end_epoch, 
              num_posts = None, 
              banned = False, 
              name = None):
    
    try:
        # Extract submissions via psaw and praw
        if num_posts is None:
        # Download all posts
            if banned:
                submissions = api_banned.search_submissions(subreddit = subreddit,
                                                 after = start_epoch,
                                                 before = end_epoch)
            else:    
                submissions = api.search_submissions(subreddit = subreddit,
                                                 after = start_epoch,
                                                 before = end_epoch)
        
        else:
        # Download specified number of posts
            if banned:
                submissions = api_banned.search_submissions(subreddit = subreddit, 
                                             limit = num_posts, 
                                             after = start_epoch,
                                             before = end_epoch)
            
            else:
                submissions = api.search_submissions(subreddit = subreddit, 
                                             limit = num_posts, 
                                             after = start_epoch,
                                             before = end_epoch)
        
        
        # Convert downloaded data to a dataframe
        if banned:
            df = pd.DataFrame([obj.d_ for obj in submissions])
            cleaned_df = df[['subreddit', 
                         'title', 
                         'selftext', 
                         'created_utc', 
                         'author',  
                         'num_comments', 
                         'score', 
                         'url']]
        else: 
            df = pd.DataFrame([vars(submission) for submission in submissions])
             # Extract the required columns
            cleaned_df = df[['subreddit', 
                         'title', 
                         'selftext', 
                         'created_utc', 
                         'author', 
                         'link_flair_text', 
                         'num_comments', 
                         'score', 
                         'upvote_ratio', 
                         'url']]
        
       
        
        # Convert dates to datetime
        cleaned_df['created_utc'] = pd.to_datetime(cleaned_df['created_utc'], unit = 's')
        
        # Convert subreddit and author objects to string 
        cleaned_df['subreddit'] = cleaned_df['subreddit'].astype(str)
        cleaned_df['author'] = cleaned_df['author'].astype(str)
        
        # Filter out removed and deleted posts
        cleaned_df = cleaned_df[(cleaned_df['selftext'] != "[removed]") | (cleaned_df['selftext'] != "[deleted]" )]
        
        if name is None:
            name = subreddit
        
        # Save as a feather - heh!
        path = 'data/' + name
        feather.write_feather(cleaned_df, path)
        print("File saved locally")
        
        # Save to storage bucket
        blob_name = name
        blob = bucket.blob(blob_name)
        source_file_name = "data/" + blob_name
        blob.upload_from_filename(source_file_name)
        print("File uploaded to {}.".format(bucket.name))
        
        return(cleaned_df)
    
    except Exception as e:
        print('Error:', e)

### Retrieve submissions for CMV

In [40]:
# Get all posts for CMV in the given time period
cmv_df = get_posts('changemyview', start_epoch, end_epoch).sort_values(by = 'score', ascending = False)
cmv_df.head(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

File saved locally
File uploaded to dulshani_bucket.


,subreddit,title,selftext,created_utc,author,link_flair_text,num_comments,score,upvote_ratio,url
45877,changemyview,CMV: Mike Bloomberg's campaign is proof that the ultra wealthy in the US can afford a higher tax rate with no ill effect on them,"Mike Bloomberg recently dropped out of the 2020 Presidential race after spending over $500 million dollars of his own money on his campaign. Even with this loss, he is still worth over $55 billion dollars.\n\n\nI believe that this effortless spending on Bloombergs part showcases the reality that is wealth inequality between the ultra rich and working class. While I do not believe the rich shou...",2020-03-05 18:22:18,boss_454,Delta(s) from OP,3753,65396,0.84,https://www.reddit.com/r/changemyview/comments/fdziov/cmv_mike_bloombergs_campaign_is_proof_that_the/
30287,changemyview,CMV: Kanye West is a shill for president Trump and running to syphon off young voters from voting for Biden.,"Kanye West is a shill for president Trump and running to syphon off young voters from voting for Biden. First, he may have an excentric take on reality but can't honestly believe that he has a chance of winning. Second, if he truly wanted to be president, he would have started his campaign last year when he still had a chance to get on state ballots. Third, he has been an avid defender of Trum...",2020-07-05 16:25:43,Psyworld,Delta(s) from OP,2080,54485,0.79,https://www.reddit.com/r/changemyview/comments/hlpd7d/cmv_kanye_west_is_a_shill_for_president_trump_and/
14962,changemyview,CMV: Every candidate for a federal position in the United States of America Government should take the same Civics test that immigrants need if they want to become citizens.,[removed],2020-11-15 18:03:51,thehistoryuniversity,Removed - Submission Rule B,1165,49222,0.86,https://www.reddit.com/r/changemyview/comments/juq86c/cmv_every_candidate_for_a_federal_position_in_the/
307,changemyview,CMV: Most Americans who oppose a national healthcare system would quickly change their tune once they benefited from it.,"I used to think I was against a national healthcare system until after I got out of the army. Granted the VA isn't always great necessarily, but it feels *fantastic* to walk out of the hospital after an appointment without ever seeing a cash register when it would have cost me potentially thousands of dollars otherwise. It's something that I don't think just veterans should be able to experien...",2021-04-27 15:30:56,CrashRiot,Delta(s) from OP,6951,44820,0.81,https://www.reddit.com/r/changemyview/comments/mzr23d/cmv_most_americans_who_oppose_a_national/
21773,changemyview,CMV: Donald Trump has not made a single lasting positive impact on the USA during his term as president.,"I write this because I am tired of the wild goose chase that is finding examples of his success. Anything surrounding Donald Trump is shrouded in divisive language and biased opinions. Liberals will have you believe he has done nothing, while conservatives will tout his unlimited success. I must be missing it, because any time I try to research into a topic I get lost in opinion and bias. \n\n...",2020-09-10 14:11:37,farm_sauce,Delta(s) from OP,5212,36281,0.74,https://www.reddit.com/r/changemyview/comments/iq41dt/cmv_donald_trump_has_not_made_a_single_lasting/


In [41]:
cmv_df.describe()

,num_comments,score,upvote_ratio
count,107759.000000,107759.000000,107759.000000
mean,36.448733,74.884548,0.768652
std,117.319998,782.396455,0.260082
min,0.000000,0.000000,0.040000
25%,0.000000,1.000000,0.550000
50%,3.000000,1.000000,0.870000
75%,39.000000,3.000000,1.000000
max,7736.000000,65396.000000,1.000000


### Retrieve submissions for Against Hate Subreddits

In [ ]:
# Get all posts for CMV since inception to current date
start_epoch_ahs = int(dt.datetime(2015, 6, 1).timestamp())
end_epoch_ahs = int(dt.datetime(2021, 5, 31).timestamp())
against_hate_df = get_posts('AgainstHateSubreddits', start_epoch_ahs, end_epoch_ahs)
against_hate_df.describe()
against_hate_df.head(5)

### Retrieve submissions for Polarised Subreddits

In [9]:
subreddits = ['unpopularopinion', 'askaconservative', 'tucker_carlson', 'chodi', 'anarcho_capitalism', 
              'averageredditor', 'femaledatingstrategy']

In [ ]:
# Retrieve submissions for subreddts
for subreddit in subreddits:
    print(subreddit)
    get_posts(subreddit)

unpopularopinion


/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning

File saved locally
File uploaded to talk-bucket.
askaconservative
File saved locally
File uploaded to talk-bucket.
tucker_carlson
File saved locally
File uploaded to talk-bucket.
chodi
File saved locally
File uploaded to talk-bucket.
anarcho_capitalism
File saved locally
File uploaded to talk-bucket.
averageredditor
File saved locally
File uploaded to talk-bucket.
femaledatingstrategy
File saved locally
File uploaded to talk-bucket.
protectandserve


In [10]:
get_posts('protectandserve')

/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to

File saved locally
File uploaded to talk-bucket.


,subreddit,title,selftext,created_utc,author,link_flair_text,num_comments,score,upvote_ratio,url
0,ProtectAndServe,How to Support Officer Down Memorial Page For Free Through Amazon Prime (IOS App Instructions),,2021-04-29 23:09:49,VanillaCrash,None,0,1,1.00,https://www.reddit.com/gallery/n1gu5d
1,ProtectAndServe,1❤️=1🙏,,2021-04-29 23:09:22,aetherconfab,None,4,1,1.00,https://i.redd.it/oshgo98g27w61.jpg
2,ProtectAndServe,Today I found out that officer Anastasios Tsakos will be having his wake and funeral in my town.,[removed],2021-04-29 21:56:55,None,Self Post,1,1,1.00,https://www.reddit.com/r/ProtectAndServe/comments/n1fbha/today_i_found_out_that_officer_anastasios_tsakos/
3,ProtectAndServe,Is this little guy a cat burglar?,[deleted],2021-04-29 20:56:15,None,None,0,1,1.00,https://i.redd.it/i8uspcdbe6w61.png
4,ProtectAndServe,Awesome photo of Counter-Terrorism in NYC.,,2021-04-29 20:39:39,Ihateallcommies,None,44,681,0.92,https://i.redd.it/yrz8f3lqb6w61.jpg
...,...,...,...,...,...,...,...,...,...,...
40288,ProtectAndServe,Fort Lauderdale Cops Arrest Woman Who Testified In Police Brutality Case,[deleted],2018-04-01 02:44:13,None,None,0,1,1.00,http://truthfight.com/fort-lauderdale-cops-take-revenge-woman-testified-police-brutality-case/
40289,ProtectAndServe,Caliber question,"Hey, I was just wondering what police departments restrict you to on firearms; i know that a lot will allow personal firearms to be carried, but do most departments have rules on what calibers they can be, and what ammunition can be used? Specifically wondering about the FN five seven.",2018-04-01 02:18:13,None,Self Post,35,5,0.78,https://www.reddit.com/r/ProtectAndServe/comments/88nmal/caliber_question/
40290,ProtectAndServe,Bodycam footage of suspect pointing gun in officer's face,,2018-04-01 01:31:09,futurecop1231,LMPD OIS,27,73,0.91,https://www.youtube.com/watch?time_continue=1&v=PahOT8HNRyE
40291,ProtectAndServe,Handy guide for SWAT signals.,,2018-04-01 00:18:05,jphuffinstuff,MEME,1,67,0.97,https://i.redd.it/65y03xqiw6p01.jpg


In [9]:
# List of required banned subreddits
banned_subreddits = ['uncensorednews', 'altright', 'Incels', 'MGTOW', 'MGTOW2', 'GenderCritical', 
                     'Physical_Removal']
banned_subreddits2 = ['donaldtrump', 'altrightchristian']

In [11]:
# Retrieve submissions for banned subreddts
for subreddit in banned_subreddits2:
    print(subreddit)
    get_posts(subreddit, banned = True)

donaldtrump


/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning

File saved locally
File uploaded to talk-bucket.
milliondollarextreme
Error: ("Connection broken: ConnectionResetError(104, 'Connection reset by peer')", ConnectionResetError(104, 'Connection reset by peer'))
altrightchristian
File saved locally
File uploaded to talk-bucket.


In [13]:
get_posts('milliondollarextreme', banned = True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

File saved locally
File uploaded to talk-bucket.


,subreddit,title,selftext,created_utc,author,num_comments,score,url
0,milliondollarextreme,"boomoopoo, you fucking snake! You've single-handedly ruined my chances with the hottest girls on the net! Those were MY good boy points you FUCK!!! Everyone else be warned: not all of us are brothers here.",,2018-09-10 19:57:01,mild_child,1,1,https://i.redd.it/mltzkscxugl11.png
1,milliondollarextreme,Take back the fatherland from (((Heinz))),,2018-09-10 19:54:41,trousersnakearmy,0,1,https://www.youtube.com/watch?v=uAO__9n0v40
2,milliondollarextreme,I love (most of) you guys,,2018-09-10 19:53:34,bigmactv,0,1,https://www.reddit.com/r/milliondollarextreme/comments/9eqgu1/i_love_most_of_you_guys/
3,milliondollarextreme,Happy Rosh Hashanah Everyone!,[removed],2018-09-10 19:49:54,ThotSlayer9000,0,1,https://www.reddit.com/r/milliondollarextreme/comments/9eqfmr/happy_rosh_hashanah_everyone/
4,milliondollarextreme,"Haha, hey guys, SO WE LOOKED AT THE DATA, anyway PayPal me. :3",,2018-09-10 19:36:40,amoonuponastick,6,1,https://i.redd.it/3a4cpazkrgl11.jpg
...,...,...,...,...,...,...,...,...
30715,milliondollarextreme,Average MDE poster,,2018-04-01 00:42:45,vendettaaff-,33,139,https://i.redd.it/krzgrt4l17p01.png
30716,milliondollarextreme,He's like a black Jew,,2018-04-01 00:35:24,startselect3,20,43,https://i.redd.it/7289nor607p01.jpg
30717,milliondollarextreme,This will be Sam in like 2-3 years,,2018-04-01 00:35:14,a2b2c2d2,2,32,https://www.youtube.com/watch?v=zR_7vWKlu6Q
30718,milliondollarextreme,am i right my fellow shekelmeisters?,,2018-04-01 00:09:54,Agrianian_Javelineer,0,1,https://i.imgur.com/ZscQI18.png


In [10]:
# The Donald: download requires to be in several phases due to large volume of data

# Set start and end dates for data retrieval
print('Phase 1')
start_epoch_donald = int(dt.datetime(2018, 4, 1).timestamp())
end_epoch_donald = int(dt.datetime(2018, 6, 30).timestamp())
d1 = get_posts('The_Donald', start_epoch_donald, end_epoch_donald, banned = True, name = 'The_Donald0')

print('Phase 2')
start_epoch_donald2 = int(dt.datetime(2019, 1, 1).timestamp())
end_epoch_donald2 = int(dt.datetime(2019, 12, 31).timestamp())
d2 = get_posts('The_Donald', start_epoch_donald2, end_epoch_donald2, banned = True, name = 'The_Donald1')

#print('Phase 3')
#start_epoch_donald3 = int(dt.datetime(2020, 1, 1).timestamp())
#end_epoch_donald3 = int(dt.datetime(2021, 5, 31).timestamp())
#d3 = get_posts('The_Donald', start_epoch_donald3, end_epoch_donald3, banned = True, name = 'The_Donald2')

Phase 1


/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning

File saved locally
File uploaded to talk-bucket.
Phase 1-2
Error: ("Connection broken: ConnectionResetError(104, 'Connection reset by peer')", ConnectionResetError(104, 'Connection reset by peer'))
Phase 2
File saved locally
File uploaded to talk-bucket.


In [10]:
print('Phase 1-2')
start_epoch_donald01 = int(dt.datetime(2018, 7, 1).timestamp())
end_epoch_donald01 = int(dt.datetime(2018, 12, 31).timestamp())
d12 = get_posts('The_Donald', start_epoch_donald01, end_epoch_donald01, banned = True, name = 'The_Donald01')

Phase 1-2


/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/conda/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning

File saved locally
File uploaded to talk-bucket.


# Retrieve User Details

### Utility Functions

In [30]:
print(df.columns.tolist())

['subreddit', 'title', 'selftext', 'created_utc', 'author', 'link_flair_text', 'num_comments', 'score', 'url']


In [ ]:
df = pd.DataFrame([ vars(post) for post in reddit.subreddit("AgainstHateSubreddits").top("year", limit=10)])
df.head(3)

In [ ]:
# Experimenting with the Pushshift API
# https://github.com/dmarx/psaw/issues/78
api = PushshiftAPI()
gen = api.search_submissions(**{"subreddit": "the_donald", "selftext:not": "[removed]|[deleted]",
                               "filter": "['title', 'selftext', 'created_utc', 'author', 'link_flair_text', 'num_comments', 'score', 'upvote_ratio', 'url']"
                               })

In [ ]:
# Group by flair, aggregate and order by count
df.groupby(['link_flair_text'])['title'].agg(['count']).sort_values(ascending = False)

### Useful Functions

### Experiment with ArchiveSort API to get aggregate user data

In [15]:
# Top 1000 posters and commenters on AskALiberal   

try:
    response = requests.get('https://archivesort.org/discuss/reddit/miser?type=comments&subreddit=AskALiberal&aggs=authors')
    response.raise_for_status()
    # access JSOn content
    jsonResponse = response.json()
    print("Entire JSON response")
    for i in jsonResponse['data']:
       print(str(i['total']) 
       + " posts have been made in subreddit: " + 
       i['_id'])
 
    print(jsonResponse)
 
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

Entire JSON response
7848 posts have been made in subreddit: [deleted]
1911 posts have been made in subreddit: reconditecache
1807 posts have been made in subreddit: ButGravityAlwaysWins
1709 posts have been made in subreddit: adeiner
1597 posts have been made in subreddit: AutoModerator
1494 posts have been made in subreddit: Hip-hop-rhino
1429 posts have been made in subreddit: polyscipaul20
1201 posts have been made in subreddit: Manoj_Malhotra
1196 posts have been made in subreddit: CTR555
1089 posts have been made in subreddit: Kakamile
1081 posts have been made in subreddit: Laniekea
992 posts have been made in subreddit: PragmaticSquirrel
936 posts have been made in subreddit: TheDismalSci
932 posts have been made in subreddit: whizzo76
869 posts have been made in subreddit: Strich-9
832 posts have been made in subreddit: Gay_Leo_Gang
831 posts have been made in subreddit: letusnottalkfalsely
810 posts have been made in subreddit: AlkalineHume
782 posts have been made in subredd